# All the basic preprocessing in one place

#### Let's apply all the preprocessing methods we have discussed so far on our Zomato dataset and see how everything works together

Fa'iq Zhafran Naufal Brinatta (220535608468 TI/22)
@author: Aman Kedia

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amankedia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amankedia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Kode tersebut mengimpor pustaka yang diperlukan untuk pemrosesan bahasa alami (NLTK), termasuk stopwords, stemming, dan lemmatization. Ini juga mengimpor pandas untuk manipulasi data dan re untuk ekspresi reguler.

In [ ]:
df = pd.read_csv("Dataset/zomato_reviews.csv")
df.head(3)

,Review,sentiment
0,Virat Kohli did a great thing to open his rest...,positive
1,This place have some really heathy options to ...,positive
2,Aerocity is the most finest place in Delhi for...,positive


In [ ]:
corpus = pd.Series(df.Review.tolist()).astype(str)

Kode ini membaca file CSV bernama "zomato_reviews.csv" dari folder "Dataset" ke dalam dataframe pandas yang disebut "df", dan kemudian menampilkan 3 baris pertama dari dataframe.

In [ ]:
corpus

0       Virat Kohli did a great thing to open his rest...
1       This place have some really heathy options to ...
2       Aerocity is the most finest place in Delhi for...
3       Yesterday evening there was small team lunch ,...
4       I find aerocity to be the best place in delhi ...
                              ...                        
1591    || DESI LANE || So we were at alipore's most h...
1592    "Desi Lane" is one of the most trending place ...
1593    One of the cool and pocket pinch restaurant at...
1594    "DESI LANE" one of the best places in town and...
1595    Looking for good place for lunch but dont wann...
Length: 1596, dtype: object

### Text Cleaning (Removal of special characters/punctuations & case folding)

In [ ]:
def text_clean(corpus, keep_list):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)

    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process

    Output : Returns the cleaned text corpus

    '''
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        for word in row.split():
            if word not in keep_list:
                p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
                p1 = p1.lower()
                qs.append(p1)
            else : qs.append(word)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus

Kode di atas membersihkan sebuah kumpulan teks (`corpus`) dengan melakukan beberapa operasi pemrosesan. Untuk setiap baris dalam `corpus`, teks dipisah menjadi kata-kata, lalu jika kata tersebut tidak ada dalam `keep_list`, kode akan menghilangkan karakter non-alfanumerik dan mengubahnya menjadi huruf kecil. Kata yang sudah diolah kemudian disatukan kembali menjadi kalimat. Jika kata ada di dalam `keep_list`, kata tersebut dibiarkan tetap. Semua baris hasil pembersihan digabungkan menjadi `cleaned_corpus` sebagai output.

### Stopwords Removal

In [ ]:
def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

Fungsi `stopwords_removal` menghapus kata-kata umum (stopwords) dari sebuah kumpulan teks (`corpus`). Fungsi ini menggunakan daftar stopwords bahasa Inggris dari pustaka NLTK, tetapi tetap mempertahankan kata-kata tanya (WH-words) seperti 'who', 'what', 'when', dll., dengan menghapusnya dari daftar stopwords. Setelah itu, setiap kata dalam kalimat yang ada di `corpus` akan dihapus jika merupakan stopword, dan teks yang telah dibersihkan dikembalikan dalam bentuk daftar kata-kata tanpa stopwords.

### Lemmatization

In [ ]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

Fungsi `lemmatize` melakukan lemmatisasi pada kumpulan teks (`corpus`). Menggunakan objek `WordNetLemmatizer` dari pustaka NLTK, fungsi ini mengubah setiap kata dalam `corpus` menjadi bentuk dasarnya (lemma), dengan menganggap setiap kata sebagai sebuah kata kerja (pos = 'v'). Hasilnya adalah kumpulan teks yang telah mengalami lemmatisasi, di mana kata-kata dikembalikan ke bentuk dasar atau kata dasarnya, seperti mengubah "running" menjadi "run".

### Stemming

In [ ]:
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

Fungsi `stem` melakukan stemming pada kumpulan teks (`corpus`), yaitu proses mengubah kata menjadi bentuk dasarnya. Fungsi ini menggunakan dua jenis stemmer: jika parameter `stem_type` disetel ke `'snowball'`, maka digunakan `SnowballStemmer` untuk bahasa Inggris. Jika tidak, fungsi menggunakan `PorterStemmer` sebagai default. Kedua stemmer ini memotong akhir kata untuk menghasilkan bentuk dasar (stem). Hasil stemming ini kemudian dikembalikan sebagai corpus yang telah diolah.

In [ ]:
def preprocess(corpus, keep_list, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)

    Input :
    'corpus' - Text corpus on which pre-processing tasks will be performed
    'keep_list' - List of words to be retained during cleaning process
    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer

    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together

    Output : Returns the processed text corpus

    '''

    if cleaning == True:
        corpus = text_clean(corpus, keep_list)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]

    return corpus

Fungsi `preprocess` melakukan serangkaian tugas pemrosesan teks pada kumpulan teks (`corpus`), seperti pembersihan, penghapusan stopwords, stemming, dan lemmatisasi. Parameter `keep_list` menentukan kata-kata yang harus dipertahankan selama pembersihan. Jika `cleaning` diaktifkan, fungsi membersihkan teks menggunakan fungsi `text_clean`. Jika `remove_stopwords` aktif, stopwords dihapus menggunakan `stopwords_removal`. Pengguna dapat memilih antara stemming (dengan opsi Porter atau Snowball stemmer) dan lemmatisasi, tetapi tidak keduanya bersamaan. Hasil akhir dikembalikan sebagai teks yang telah diproses.

In [ ]:
common_dot_words = ['U.S.A', 'Mr.', 'Mrs.', 'D.C.']

In [ ]:
# Preprocessing with Lemmatization here
corpus_with_lemmatization = preprocess(corpus, keep_list = common_dot_words, stemming = False, stem_type = None, lemmatization = True, remove_stopwords = True)

In [ ]:
# Preprocessing with Stemming here here
corpus_with_stemming = preprocess(corpus, keep_list = common_dot_words, stemming = True, stem_type = "snowball", lemmatization = False, remove_stopwords = True)

Pada kode di atas, dua preprocessing dilakukan pada kumpulan teks (`corpus`). Yang pertama, `corpus_with_lemmatization`, menggunakan fungsi `preprocess` untuk menerapkan lemmatisasi dengan mempertahankan kata-kata tertentu yang mengandung titik (seperti 'U.S.A', 'Mr.', 'Mrs.', 'D.C.') dan menghapus stopwords, tanpa menggunakan stemming. Yang kedua, `corpus_with_stemming` juga memproses `corpus` tetapi dengan menggunakan stemming (menggunakan Snowball stemmer) dan tetap menghapus stopwords, sambil tidak menerapkan lemmatisasi. Kedua hasil dipisahkan ke dalam variabel yang berbeda untuk analisis lebih lanjut.

# Let's see the results on applying

### 1. Lemmatization
### 2. Stemming

Note: Stopwords removal and text cleaning have been applied on both the occassions.

In [ ]:
print("Original string: ", corpus[0])

Original string:  Virat Kohli did a great thing to open his restaurant in an exquisite place of Delhi. Wide range of food with lots and lots of options on drinks. Courteous staff with a quick response on anything.


In [ ]:
print("String after lemmatiization: ", corpus_with_lemmatization[0])

String after lemmatiization:  virat kohli great thing open restaurant exquisite place delhi wide range food lot lot options drink courteous staff quick response anything


In [ ]:
print("String after stemming: ", corpus_with_stemming[0])

String after stemming:  virat koh great thing open restaur exquisit place delhi wide rang food lot lot option drink courteous staff quick respons anyth


Perbandingan antara string asli dan hasil lemmatisasi serta stemming menunjukkan perbedaan signifikan dalam keterbacaan dan makna. Lemmatisasi mempertahankan bentuk dasar kata dengan lebih baik, seperti "options" menjadi "option", sehingga konteks dan nuansa kalimat tetap jelas. Sebaliknya, stemming mengubah kata-kata menjadi bentuk yang lebih pendek dan sering kali tidak sesuai, seperti "courteous" menjadi "courtes" dan "response" menjadi "respons", yang dapat mengaburkan makna. Secara keseluruhan, lemmatisasi lebih efektif dalam menjaga makna dan keterbacaan, sementara stemming lebih fokus pada penyederhanaan tanpa mempertimbangkan konteks.